In [5]:
import papermill as pm
import pandas as pd
import time
from pathlib import Path
import sys

In [6]:
instant = time.strftime("%Y-%m-%d_%H:%M:%S", time.localtime())

In [7]:
# Parameters
nb_name = "3_elec_out"
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
cliente = "Generic"
estim_year = 2025
estim_month = 6
issue_year = 2025
issue_month = 6
operation = "verif"
update_contract_data = True
api_call = False
ssaa_dn = False
path_alias_elec = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/alias_clientes_elec.xlsx"
path_omie_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/omie_matrix_verif.pkl"
path_load_parquet = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/elec_load.parquet"
db_elec_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec.db"
db_elec_manual_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec_manual.pkl"
path_ver_template = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/templates/Template 2024.xlsx"
path_estim_template = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/templates/Estimacion_fra.xlsx"
path_peajes_cargos_energia = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/peajes_cargos_energia.xlsx"
path_peajes_energia = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/peajes_energia.xlsx"
path_peajes_potencia = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/peajes_potencia.xlsx"
path_cargos_energia = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/cargos_energia.xlsx"
path_cargos_potencia = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/cargos_potencia.xlsx"
path_estructura_cargos = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/estructura_cargos.xlsx"
path_dto_peajes_electrointensivos = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/dto_peajes_electrointensivos.xlsx"
path_ie = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/ie.xlsx"
path_coefs_exc = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/regulation/elec/coefs_excesos.xlsx"
nb_read_contracts = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/4a_read_contract.ipynb"
nb_read_contracts_out = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/outputs/verification_project/notebooks/4a_read_contract_out.ipynb"
nb_contract_market_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/4b_merge_contract_omie.ipynb"
nb_contract_market_matrix_out = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/outputs/verification_project/notebooks/4b_merge_contract_omie_out.ipynb"
nb_invoices = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/5_hunt_invoices.ipynb"
nb_invoices_out = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/outputs/verification_project/notebooks/5_hunt_invoices_out.ipynb"
nb_estim = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/5c_estimacion.ipynb"
nb_estim_out = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/outputs/verification_project/notebooks/5c_estimacion_out.ipynb"
nb_verif = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/5b_ver_notebook.ipynb"


In [8]:
import utils.utils_var as utils_var
from config.config_loader import Config

config = Config(config_file=config_path)


In [ ]:
from utils.logging_setup import get_logger

logger = get_logger(name=f"{nb_name} {cliente}")
logger.info(f"Starting verification process for {cliente} on {instant}")

In [121]:
alias_df = pd.read_excel(path_alias_elec)
alias_df_fil = alias_df[alias_df['cliente'] == cliente]
if cliente == 'PR':
    alias_df_fil = alias_df_fil[
                                # (alias_df_fil['alias'].isin(['HQ1','HQ2','HQ3']))
                                (~alias_df_fil['alias'].isin(['VELERIN'])) &
                                # (~alias_df_fil['alias'].isin(['WARNER BS'])) &
                                (~alias_df_fil['alias'].isin(['WARNER BS','WARNER DW','WARNER BQ'])) &
                                # (alias_df_fil['alias'].isin(['WARNER BS','WARNER DW'])) &
                                # (alias_df_fil['alias'].isin(['WARNER BQ','CULLERA'])) &
                                ((alias_df_fil['cups'] != 'ES0022000009138571BZ'))
                                # (alias_df_fil['alias'].isin(['FAUNIA']))
                                ]
    # alias_df_fil = alias_df_fil[((alias_df_fil['alias'] == 'AQUAPARK'))]

if cliente == 'Grupo Inspired':
    alias_df_fil = alias_df_fil[
                                # ((alias_df_fil['tarifa'] == '6.1 TD'))
                                (alias_df_fil['alias'].isin(['KC SOTO VIÑUELAS 1']))
                                # (alias_df_fil['alias'].isin(['MIRABAL']))
                                ]
logger.info("Completed supply point filtering")

           cliente               alias                  cups  tarifa  \
25  Grupo Inspired  KC SOTO VIÑUELAS 1  ES0021000005776632EH  6.1 TD   

    electrointensivo  contrato  
25             False         1  


In [122]:
for index, row in alias_df_fil.iterrows():

    contract_id = row['contrato']
    tarifa = row['tarifa']
    cups= row['cups']
    electrointensivo = row['electrointensivo']
    alias = row['alias']

    output_estim_excel =  \
    config.get_path(
        "outputs.verification_project.estim.cliente.output_estim_excel", customer_id=cliente, alias=alias, month=estim_month, year=estim_year
        )
    output_estim_pdf =  \
    config.get_path(
        "outputs.verification_project.estim.cliente.output_estim_pdf", customer_id=cliente, alias=alias, month=estim_month, year=estim_year
        )
    
    path_datos_cliente = \
    config.get_path(
        "data_path.elec.customers.customer_single.customer_data", customer_id=cliente, contract_id=contract_id
        )

    path_contract_matrix = \
    config.get_path(
        "data_path.elec.customers.customer_single.raw.contract_matrix", cups=cups, customer_id=cliente
        )
    path_master_matrix = \
    config.get_path(
        "data_path.elec.customers.customer_single.raw.master_matrix", customer_id=cliente, cups=cups
        )
    path_coberturas_df = \
    config.get_path("data_path.elec.customers.customer_single.raw.df_coberturas", customer_id=cliente, contract_id=contract_id)

    if update_contract_data:
        logger.info(f'Retrieving contract info for {alias}...')
        pm.execute_notebook(
            nb_read_contracts, nb_read_contracts_out,##########
            parameters=dict(
                nb_name = Path(nb_read_contracts_out).stem,
                config_path = config_path,
                cliente=cliente,
                tarifa = tarifa,
                contract_id = contract_id,
                cups= cups,
                electrointensivo = electrointensivo,
                path_datos_cliente = path_datos_cliente,
                path_peajes_cargos_energia = path_peajes_cargos_energia,
                path_peajes_energia = path_peajes_energia,
                path_peajes_potencia = path_peajes_potencia,
                path_cargos_energia = path_cargos_energia,
                path_cargos_potencia = path_cargos_potencia,
                path_estructura_cargos = path_estructura_cargos,
                path_dto_peajes_electrointensivos = path_dto_peajes_electrointensivos,
                path_ie = path_ie,
                path_coberturas_df = path_coberturas_df,
                path_contract_matrix = path_contract_matrix,
                path_coefs_exc = path_coefs_exc
            ),
        log_output=True
        )
    else:
        logger.info(f'Skipping contract info retrieval for {alias}...')

    logger.info(f'Building master matrix for {alias}...')
    pm.execute_notebook(
        nb_contract_market_matrix, nb_contract_market_matrix_out,##############
        parameters=dict(
            nb_name = Path(nb_contract_market_matrix_out).stem,
            config_path = config_path,
            tarifa = tarifa,
            path_contract_matrix = path_contract_matrix,
            path_omie_matrix = path_omie_matrix,
            path_master_matrix = path_master_matrix,
        ),
        log_output=True
    )
    logger.info(f'Built master hourly matrix for alias {alias}')
    if operation == 'verif':
        
        logger.info(f'Looking for invoices for {alias}...')
        pm.execute_notebook(
            nb_invoices, nb_invoices_out,##############
            parameters=dict(
                nb_name = Path(nb_invoices_out).stem,
                config_path = config_path,
                nb_verif = nb_verif,
                issue_year = issue_year,
                issue_month = issue_month,
                cliente=cliente,
                alias = alias,
                tarifa = tarifa,
                cups= cups,
                path_load_parquet = path_load_parquet,
                path_contract_matrix = path_contract_matrix,
                path_coberturas_df = path_coberturas_df,
                path_master_matrix = path_master_matrix,
                db_elec_path = db_elec_path,
                db_elec_manual_path = db_elec_manual_path,
                template_path = path_ver_template
            ),
        stdout_file=sys.stdout,
        stderr_file=sys.stderr,
        log_output=True
        )
    elif operation == 'estim':

        logger.info(f'Estimating cost for {utils_var.map_month_cap(estim_month)} {estim_year}...')
        pm.execute_notebook(
            
            nb_estim, nb_estim_out,##############
            parameters=dict(
                nb_name = Path(nb_estim_out).stem,
                config_path = config_path,
                nb_verif = nb_verif,
                estim_year = estim_year,
                estim_month = estim_month,
                cliente=cliente,
                alias = alias,
                tarifa = tarifa,
                cups= cups,
                path_load_parquet = path_load_parquet,
                path_contract_matrix = path_contract_matrix,
                path_coberturas_df = path_coberturas_df,
                path_master_matrix = path_master_matrix,
                db_elec_path = db_elec_path,
                db_elec_manual_path = db_elec_manual_path,
                template_path = path_estim_template,
                output_estim_excel = output_estim_excel,
                output_estim_pdf = output_estim_pdf
            ),
            stdout_file=sys.stdout,
            stderr_file=sys.stderr,
            log_output=True
        )

Retrieving contract info for KC SOTO VIÑUELAS 1...


Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Building master matrix for KC SOTO VIÑUELAS 1...


Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Looking for invoices for KC SOTO VIÑUELAS 1...


Executing:   0%|          | 0/8 [00:00<?, ?cell/s]